In [1]:
!pip install elasticsearch

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import os
from subprocess import Popen, PIPE, STDOUT
from elasticsearch import Elasticsearch
import elasticsearch.helpers

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from transformers import pipeline

In [3]:
#Setting up ElasticSearch

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
!curl -X GET "localhost:9200/"
es = Elasticsearch()
es.ping()

{
  "name" : "4366206183fe",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "J6CD9NBiRaSfUy4zT-FDbA",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


True

In [4]:
def read_data(source_file):
    
    data = pd.read_csv(source_file,index_col=0)
    data = data.fillna(0)
    
    return data

In [5]:
def insert_data(data, es, this_index):

    for row_id, row in data.iterrows():

        doc = row.to_dict()
        es.index(index= this_index, id= row_id, body= doc)
    

In [12]:
def scan_full_database(es, this_index):
    
    documents = elasticsearch.helpers.scan(es,
        index= this_index,
        preserve_order=True,
        query={"query": {"match_all": {}}},
    )
    
    tweets = []
    
    for document in documents:
        cleaned_tweet = clean_text(document["_source"]["tweet"])
        tweets.append(cleaned_tweet)
    
    return tweets


In [5]:
def clean_text(text):
    
    text = re.sub('https?://\S+|www\.\S+', '', text) #Remove URL
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #Remove punctuation
    
    return text

In [6]:
def NER_model(text, ner):
    
    #ner = pipeline('ner', grouped_entities=True)
    output = ner(text)
    
    entities = []
    print(output)
    for instance in output:
        entity_tuple = (instance['entity_group'], instance['word'])
        entities.append(entity_tuple)
    
    print(entities)
    return entities
    #doc = nlp(text)
    #return doc.ents

In [16]:

def get_entities (tweets, ner):
    
    entity_dictionary = {}
    
    for tweet in tweets:
        
        entities = NER_model(tweet, ner)
        
        for entity in entities:
            
            if (entity[0] not in entity_dictionary):
                entity_dictionary[entity[0]] = {}
                
            if (entity[1] not in entity_dictionary[entity[0]]):
                entity_dictionary[entity[0]][entity[1]] = 0
                
            entity_dictionary[entity[0]][entity[1]] += 1
            
    return entity_dictionary
        

In [20]:
def show_wordcloud(dictionary):
    wordcloud = WordCloud(max_words=50,normalize_plurals=False,background_color='white').generate_from_frequencies(dictionary)

    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [12]:
data = read_data("/kaggle/input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv")
insert_data(data, es, "trump_tweets")


In [13]:
tweets = scan_full_database(es, "trump_tweets")

In [22]:
ner = pipeline('ner', grouped_entities=True)
entity_dictionary = get_entities(tweets[:1000], ner)

[{'entity_group': 'PER', 'score': 0.9986373037099838, 'word': 'Thomas Frieden', 'start': 29, 'end': 43}, {'entity_group': 'ORG', 'score': 0.9980486333370209, 'word': 'CDC', 'start': 47, 'end': 50}, {'entity_group': 'LOC', 'score': 0.971662312746048, 'word': 'West Africa', 'start': 118, 'end': 129}]
[('PER', 'Thomas Frieden'), ('ORG', 'CDC'), ('LOC', 'West Africa')]
[{'entity_group': 'PER', 'score': 0.9986373037099838, 'word': 'Thomas Frieden', 'start': 29, 'end': 43}, {'entity_group': 'ORG', 'score': 0.9980486333370209, 'word': 'CDC', 'start': 47, 'end': 50}, {'entity_group': 'LOC', 'score': 0.971662312746048, 'word': 'West Africa', 'start': 118, 'end': 129}]
[('PER', 'Thomas Frieden'), ('ORG', 'CDC'), ('LOC', 'West Africa')]
[{'entity_group': 'LOC', 'score': 0.9991544485092163, 'word': 'US', 'start': 12, 'end': 14}, {'entity_group': 'MISC', 'score': 0.9249923676252365, 'word': 'AMERICA', 'start': 27, 'end': 34}]
[('LOC', 'US'), ('MISC', 'AMERICA')]
[{'entity_group': 'PER', 'score': 0.

[{'entity_group': 'ORG', 'score': 0.9219020009040833, 'word': 'NYD', 'start': 0, 'end': 3}, {'entity_group': 'ORG', 'score': 0.7011495232582092, 'word': '##yN', 'start': 6, 'end': 8}, {'entity_group': 'PER', 'score': 0.9908110976219178, 'word': 'Mort Zuckerman', 'start': 51, 'end': 65}]
[('ORG', 'NYD'), ('ORG', '##yN'), ('PER', 'Mort Zuckerman')]
[{'entity_group': 'LOC', 'score': 0.9997247159481049, 'word': 'Saudi Arabia', 'start': 0, 'end': 12}, {'entity_group': 'LOC', 'score': 0.999638706445694, 'word': 'United States', 'start': 34, 'end': 47}, {'entity_group': 'ORG', 'score': 0.8380170265833536, 'word': 'AlWale', 'start': 113, 'end': 119}, {'entity_group': 'ORG', 'score': 0.5404005646705627, 'word': '##bin', 'start': 121, 'end': 124}]
[('LOC', 'Saudi Arabia'), ('LOC', 'United States'), ('ORG', 'AlWale'), ('ORG', '##bin')]
[{'entity_group': 'MISC', 'score': 0.9987146258354187, 'word': 'Mexican', 'start': 11, 'end': 18}, {'entity_group': 'LOC', 'score': 0.9997982978820801, 'word': 'Me

[{'entity_group': 'ORG', 'score': 0.8876114785671234, 'word': 'Macys', 'start': 47, 'end': 52}]
[('ORG', 'Macys')]
[{'entity_group': 'ORG', 'score': 0.9981266260147095, 'word': 'Univision', 'start': 34, 'end': 43}]
[('ORG', 'Univision')]
[{'entity_group': 'ORG', 'score': 0.9981266260147095, 'word': 'Univision', 'start': 34, 'end': 43}]
[('ORG', 'Univision')]
[{'entity_group': 'ORG', 'score': 0.9981266260147095, 'word': 'Univision', 'start': 34, 'end': 43}]
[('ORG', 'Univision')]
[{'entity_group': 'LOC', 'score': 0.7186957597732544, 'word': 'Mexico', 'start': 0, 'end': 6}, {'entity_group': 'LOC', 'score': 0.9995744824409485, 'word': 'USA', 'start': 72, 'end': 75}]
[('LOC', 'Mexico'), ('LOC', 'USA')]
[{'entity_group': 'PER', 'score': 0.5581575036048889, 'word': 'Chuck', 'start': 11, 'end': 16}, {'entity_group': 'ORG', 'score': 0.4356880486011505, 'word': '##T', 'start': 16, 'end': 17}, {'entity_group': 'ORG', 'score': 0.9460375110308329, 'word': 'Meet The Press', 'start': 32, 'end': 46}]

[{'entity_group': 'PER', 'score': 0.8798784464597702, 'word': 'Lawrence O ’ Donnell', 'start': 6, 'end': 24}, {'entity_group': 'MISC', 'score': 0.8015155792236328, 'word': 'A', 'start': 85, 'end': 86}, {'entity_group': 'MISC', 'score': 0.9540492296218872, 'word': '##ice', 'start': 92, 'end': 95}]
[('PER', 'Lawrence O ’ Donnell'), ('MISC', 'A'), ('MISC', '##ice')]
[{'entity_group': 'LOC', 'score': 0.9995904266834259, 'word': 'Las Vegas', 'start': 18, 'end': 27}, {'entity_group': 'ORG', 'score': 0.8546401560306549, 'word': 'Hokey', 'start': 58, 'end': 63}, {'entity_group': 'LOC', 'score': 0.9820648729801178, 'word': 'New York', 'start': 72, 'end': 80}]
[('LOC', 'Las Vegas'), ('ORG', 'Hokey'), ('LOC', 'New York')]
[{'entity_group': 'LOC', 'score': 0.9995904266834259, 'word': 'Las Vegas', 'start': 18, 'end': 27}, {'entity_group': 'ORG', 'score': 0.8546401560306549, 'word': 'Hokey', 'start': 58, 'end': 63}, {'entity_group': 'LOC', 'score': 0.9820648729801178, 'word': 'New York', 'start': 72

[{'entity_group': 'ORG', 'score': 0.924489608832768, 'word': 'BreitbartNews', 'start': 0, 'end': 13}, {'entity_group': 'PER', 'score': 0.9937971035639445, 'word': 'Steve Bannon', 'start': 72, 'end': 84}]
[('ORG', 'BreitbartNews'), ('PER', 'Steve Bannon')]
[{'entity_group': 'ORG', 'score': 0.9995632767677307, 'word': 'Des Moines Register', 'start': 27, 'end': 46}, {'entity_group': 'LOC', 'score': 0.9993723034858704, 'word': 'Iowa', 'start': 92, 'end': 96}]
[('ORG', 'Des Moines Register'), ('LOC', 'Iowa')]
[{'entity_group': 'ORG', 'score': 0.9995632767677307, 'word': 'Des Moines Register', 'start': 27, 'end': 46}, {'entity_group': 'LOC', 'score': 0.9993723034858704, 'word': 'Iowa', 'start': 92, 'end': 96}]
[('ORG', 'Des Moines Register'), ('LOC', 'Iowa')]
[{'entity_group': 'ORG', 'score': 0.9995632767677307, 'word': 'Des Moines Register', 'start': 27, 'end': 46}, {'entity_group': 'LOC', 'score': 0.9993723034858704, 'word': 'Iowa', 'start': 92, 'end': 96}]
[('ORG', 'Des Moines Register'),

...

[{'entity_group': 'ORG', 'score': 0.9395189881324768, 'word': 'SarahPalinUSA', 'start': 39, 'end': 52}, {'entity_group': 'PER', 'score': 0.9969736337661743, 'word': 'Cruz', 'start': 70, 'end': 74}, {'entity_group': 'PER', 'score': 0.9917673468589783, 'word': 'Glenn', 'start': 97, 'end': 102}]
[('ORG', 'SarahPalinUSA'), ('PER', 'Cruz'), ('PER', 'Glenn')]
[{'entity_group': 'ORG', 'score': 0.9395189881324768, 'word': 'SarahPalinUSA', 'start': 39, 'end': 52}, {'entity_group': 'PER', 'score': 0.9969736337661743, 'word': 'Cruz', 'start': 70, 'end': 74}, {'entity_group': 'PER', 'score': 0.9917673468589783, 'word': 'Glenn', 'start': 97, 'end': 102}]
[('ORG', 'SarahPalinUSA'), ('PER', 'Cruz'), ('PER', 'Glenn')]
[{'entity_group': 'ORG', 'score': 0.9395189881324768, 'word': 'SarahPalinUSA', 'start': 39, 'end': 52}, {'entity_group': 'PER', 'score': 0.9969736337661743, 'word': 'Cruz', 'start': 70, 'end': 74}, {'entity_group': 'PER', 'score': 0.9917673468589783, 'word': 'Glenn', 'start': 97, 'end': 

[{'entity_group': 'PER', 'score': 0.9976552724838257, 'word': 'Cruz', 'start': 0, 'end': 4}, {'entity_group': 'ORG', 'score': 0.9687726696332296, 'word': 'TARP', 'start': 22, 'end': 26}, {'entity_group': 'ORG', 'score': 0.9992007911205292, 'word': 'Goldman Sachs', 'start': 52, 'end': 65}]
[('PER', 'Cruz'), ('ORG', 'TARP'), ('ORG', 'Goldman Sachs')]
[{'entity_group': 'PER', 'score': 0.9976478815078735, 'word': 'Cruz', 'start': 3, 'end': 7}, {'entity_group': 'LOC', 'score': 0.850296139717102, 'word': 'Wall St', 'start': 30, 'end': 37}, {'entity_group': 'ORG', 'score': 0.913582369685173, 'word': 'Goldman SachsCiti', 'start': 52, 'end': 69}]
[('PER', 'Cruz'), ('LOC', 'Wall St'), ('ORG', 'Goldman SachsCiti')]
[{'entity_group': 'LOC', 'score': 0.9997877478599548, 'word': 'Iowa', 'start': 8, 'end': 12}]
[('LOC', 'Iowa')]
[{'entity_group': 'LOC', 'score': 0.9997877478599548, 'word': 'Iowa', 'start': 8, 'end': 12}]
[('LOC', 'Iowa')]
[{'entity_group': 'PER', 'score': 0.8969218333562216, 'word': 

[{'entity_group': 'ORG', 'score': 0.5111605525016785, 'word': '##ob', 'start': 1, 'end': 3}, {'entity_group': 'PER', 'score': 0.7760142683982849, 'word': '##derp', 'start': 6, 'end': 10}, {'entity_group': 'ORG', 'score': 0.4460253119468689, 'word': '##la', 'start': 10, 'end': 12}, {'entity_group': 'PER', 'score': 0.9951380491256714, 'word': 'Cruz', 'start': 127, 'end': 131}]
[('ORG', '##ob'), ('PER', '##derp'), ('ORG', '##la'), ('PER', 'Cruz')]
[{'entity_group': 'ORG', 'score': 0.5111605525016785, 'word': '##ob', 'start': 1, 'end': 3}, {'entity_group': 'PER', 'score': 0.7760142683982849, 'word': '##derp', 'start': 6, 'end': 10}, {'entity_group': 'ORG', 'score': 0.4460253119468689, 'word': '##la', 'start': 10, 'end': 12}, {'entity_group': 'PER', 'score': 0.9951380491256714, 'word': 'Cruz', 'start': 127, 'end': 131}]
[('ORG', '##ob'), ('PER', '##derp'), ('ORG', '##la'), ('PER', 'Cruz')]
[{'entity_group': 'ORG', 'score': 0.5111605525016785, 'word': '##ob', 'start': 1, 'end': 3}, {'entity_

[{'entity_group': 'PER', 'score': 0.955711019039154, 'word': 'Mort Zuckerman', 'start': 6, 'end': 20}, {'entity_group': 'ORG', 'score': 0.9952357610066732, 'word': 'NYDailyNews', 'start': 44, 'end': 55}, {'entity_group': 'ORG', 'score': 0.9823870658874512, 'word': 'Paper', 'start': 88, 'end': 93}]
[('PER', 'Mort Zuckerman'), ('ORG', 'NYDailyNews'), ('ORG', 'Paper')]
[{'entity_group': 'PER', 'score': 0.955711019039154, 'word': 'Mort Zuckerman', 'start': 6, 'end': 20}, {'entity_group': 'ORG', 'score': 0.9952357610066732, 'word': 'NYDailyNews', 'start': 44, 'end': 55}, {'entity_group': 'ORG', 'score': 0.9823870658874512, 'word': 'Paper', 'start': 88, 'end': 93}]
[('PER', 'Mort Zuckerman'), ('ORG', 'NYDailyNews'), ('ORG', 'Paper')]
[{'entity_group': 'ORG', 'score': 0.9522542258103689, 'word': 'NYDailyNews', 'start': 10, 'end': 21}, {'entity_group': 'PER', 'score': 0.9492356657981873, 'word': 'Mort Zuckerman', 'start': 34, 'end': 48}]
[('ORG', 'NYDailyNews'), ('PER', 'Mort Zuckerman')]
[{'e

[{'entity_group': 'PER', 'score': 0.9994674324989319, 'word': 'Ted Cruz', 'start': 0, 'end': 8}, {'entity_group': 'MISC', 'score': 0.8580148220062256, 'word': 'Bible', 'start': 35, 'end': 40}]
[('PER', 'Ted Cruz'), ('MISC', 'Bible')]
[{'entity_group': 'PER', 'score': 0.9803362488746643, 'word': 'JebBush', 'start': 8, 'end': 15}, {'entity_group': 'PER', 'score': 0.9992272555828094, 'word': 'Hillary Clinton', 'start': 21, 'end': 36}, {'entity_group': 'MISC', 'score': 0.3770204782485962, 'word': 'GO', 'start': 72, 'end': 74}, {'entity_group': 'ORG', 'score': 0.7736040552457174, 'word': '##PDebate', 'start': 74, 'end': 81}]
[('PER', 'JebBush'), ('PER', 'Hillary Clinton'), ('MISC', 'GO'), ('ORG', '##PDebate')]
[{'entity_group': 'PER', 'score': 0.9940544366836548, 'word': 'Ted', 'start': 38, 'end': 41}, {'entity_group': 'ORG', 'score': 0.8438664674758911, 'word': 'Blaze', 'start': 54, 'end': 59}]
[('PER', 'Ted'), ('ORG', 'Blaze')]
[{'entity_group': 'ORG', 'score': 0.7959526181221008, 'word':

In [25]:
for key in entity_dictionary:
    
    print(key)
    show_wordcloud(entity_dictionary[key])
    
    